In [ ]:
import io
from google.colab import drive, files
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
auth.authenticate_user()  # verify your account to read files which you have access to. Make sure you have permission to read the file!
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

!pip install transformers
!pip install nlp
!pip install sentencepiece
!pip install emoji
!pip install pytorch_lightning
import transformers
from transformers import AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 30.6 MB/s 
     |████████████████████████████████| 596 kB 59.6 MB/s 
     |████████████████████████████████| 6.6 MB 61.8 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 28.4 MB/s 
     |████████████████████████████████| 212 kB 71.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 26.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 34

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To mount in a folder of google drive

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Labellisation data personnels long covid.xlsx - Tweets (1).csv to Labellisation data personnels long covid.xlsx - Tweets (1).csv
Saving Labellisation data personnels long covid.xlsx - Reddit (1).csv to Labellisation data personnels long covid.xlsx - Reddit (1).csv


In [ ]:
#Prepare Twitter Data
df_tweets = pd.read_csv("Labellisation data personnels long covid.xlsx - Tweets (1).csv")
df_tweets = df_tweets[:696]
df_tweets = df_tweets[["text","Personal (1=yes, 0=no) Hanin"]].rename(columns = {'Personal (1=yes, 0=no) Hanin':'Labels'})

In [ ]:
#Prepare Reddit Data
df_reddit = pd.read_csv("Labellisation data personnels long covid.xlsx - Reddit (1).csv")
df_reddit = df_reddit[:700]
df_reddit = df_reddit[["text",'Personal (1=yes, 0=no) Charline']].rename(columns = {'Personal (1=yes, 0=no) Charline':'Labels'})

In [ ]:
#Merge both datasets
df = pd.concat([df_tweets, df_reddit])
df = df.reset_index(drop=True)
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df = df.dropna()
print(type(df["Labels"][0]),type(df["Labels"][1]))
df = df.astype({"Labels": int})

<class 'str'> <class 'str'>


In [ ]:
#Check labels distribution
print(df.Labels.value_counts())

0    993
1    402
Name: Labels, dtype: int64


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

**Get a file from Google Drive in pandas **

TWEET EXAMPLE

In [ ]:
# Normalisation for BertTweet
from nltk.tokenize import TweetTokenizer
from emoji import demojize
import re

tokenizer = TweetTokenizer()

# https://huggingface.co/vinai/bertweet-base
def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):

    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

FROM HUGGINGFACE: https://huggingface.co/transformers/training.html

In [ ]:
print(tokenizer.tokenize(normalizeTweet(df.iloc[0]["text"])))
df.head()

['@USER', 'No', ',', 'just', 'no', '.', 'What', "'", 's', 'this', 'obsession', 'with', 'exercise', 'as', 'a', 'cure-all', '?', 'It', 'could', 'even', 'be', 'dangerous', ',', 'given', 'that', 'many', '#LongCovid', 'patients', 'suffer', 'from', 'post-exertional', 'malaise', '#PEM', '.', 'For', 'some', 'even', 'the', 'smallest', 'activity', 'can', 'make', 'them', 'irrevocably', 'worse', ',', 'sometimes', 'permanently', '.', '#StopRestPace']


,text,Labels
0,"@NeuroscienceNew No, just no. What's this obse...",0
1,Just wanted to share my story to see if anyone...,0
2,@1marzipan And should one assume that every we...,0
3,"When I got covid, the only underlying conditio...",0
4,"(24, M) I have always lived a healthy life, no...",0


In [ ]:
from transformers import BertForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
#from nlp import load_dataset
from transformers import DistilBertTokenizerFast
import torch

data = df #df_old, #df2
text = data["text"].map(normalizeTweet).values.tolist()
labels = data["Labels"].values.tolist()
train_texts, test_texts, train_labels, test_labels = train_test_split(text, labels, test_size=0.33)
train_texts, val_texts, train_labels, val_labels = train_test_split(text, labels, test_size=0.2)
print("Train: {}".format(len(train_texts)))
print("Val: {}".format(len(val_texts)))
print("Test: {}".format(len(test_texts)))
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# truncation, padding = true ensures that all sentences are padded to the same length and are truncated to be no longer model's max input lengts
# => allows to feed batches of sequences 
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
print(len(train_encodings))
print(len(test_encodings))
print(len(val_encodings))
print(val_encodings.keys())
#model = BertForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2) # create BERT Model with weights and randomly initializes sequence classification head with output size  2
#model.train() # puts model in train mode

Train: 1116
Val: 279
Test: 461


Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


3
3
3
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
# Transform labels + encodings into Pytorch DataSet object (including __len__, __getitem__)
class TweetDataSet(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
      
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataSet(train_encodings, train_labels)
val_dataset = TweetDataSet(val_encodings, val_labels)
test_dataset = TweetDataSet(test_encodings, test_labels)
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))


1116
279
461


Now we have datasets. Two ways to fine-tune model: 
1) Using Trainer
2) Native PyTorch

In [ ]:
# 1) Trainer 
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
CUDA_LAUNCH_BLOCKING=1
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10
)

# AutoModelForSequenceClassification adds a fully connected layer after BERT
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

Step,Training Loss
10,0.686900
20,0.666400
30,0.641000
40,0.618200
50,0.592600
60,0.589400
70,0.536000
80,0.477000
90,0.451300
100,0.397400




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=180, training_loss=0.4181286205848058, metrics={'train_runtime': 225.3956, 'train_samples_per_second': 49.513, 'train_steps_per_second': 0.799, 'total_flos': 734079844454400.0, 'train_loss': 0.4181286205848058, 'epoch': 10.0})

In [ ]:
eval_output = trainer.evaluate(test_dataset) 
print(eval_output)

***** Running Evaluation *****
  Num examples = 461
  Batch size = 32


{'eval_loss': 0.15992209315299988, 'eval_accuracy': 0.9457700650759219, 'eval_f1': 0.9146757679180887, 'eval_precision': 0.881578947368421, 'eval_recall': 0.950354609929078, 'eval_runtime': 3.3128, 'eval_samples_per_second': 139.159, 'eval_steps_per_second': 4.528, 'epoch': 10.0}


Save fine-tuned model 

In [ ]:
trainer.save_model("drive/MyDrive/Data_Files/Fine-tuned_models/Own_long_covid_classifier_06072022")

Saving model checkpoint to drive/MyDrive/Data_Files/Fine-tuned_models/Own_long_covid_classifier_06072022
Configuration saved in drive/MyDrive/Data_Files/Fine-tuned_models/Own_long_covid_classifier_06072022/config.json
Model weights saved in drive/MyDrive/Data_Files/Fine-tuned_models/Own_long_covid_classifier_06072022/pytorch_model.bin
